In [1]:
import requests
import time
import datetime
import sys
from azure.cognitiveservices.vision.computervision.models import TextOperationStatusCodes
import os
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
from PIL import Image
import imageio
import time
import glob
import re
#from pdf2image import convert_from_path, convert_from_bytes
from collections import Counter 
import csv
from sys import argv

In [527]:
def OCR_v1(image_stream):

    COMPUTERVISION_LOCATION = os.environ.get("COMPUTERVISION_LOCATION", "centralindia")

    subscription_key = "697d2b5119b544a6bab090c849809baa"



    endpoint="https://ocr-computervision-innovation.cognitiveservices.azure.com/",
    credentials=CognitiveServicesCredentials(subscription_key)


    ocr_url = endpoint[0] + "vision/v2.1/read/core/asyncBatchAnalyze"

    
    start=datetime.datetime.now()

    try:
    

            # Set image_url to the URL of an image that you want to analyze.
            headers = {'Ocp-Apim-Subscription-Key': subscription_key, 'Content-Type': 'application/octet-stream'}
            params = {'language': 'en', 'detectOrientation': 'true'}
            response = requests.post(ocr_url, headers=headers,params=params, data=image_stream)
            


            response.raise_for_status()

            # Extracting text requires two API calls: One call to submit the
            # image for processing, the other to retrieve the text found in the image.

            # Holds the URI used to retrieve the recognized text.
            operation_url = response.headers["Operation-Location"]

            # The recognized text isn't immediately available, so poll to wait for completion.
            analysis = {}
            poll = True
            while (poll):
                response_final = requests.get(
                    response.headers["Operation-Location"], headers=headers)
                analysis = response_final.json()
                time.sleep(1)
                if ("recognitionResults" in analysis):
                    poll = False
                if ("status" in analysis and analysis['status'] == 'Failed'):
                    poll = False

            polygons = []
            text=[]

            if ("recognitionResults" in analysis):
                # Extract the recognized text, with bounding boxes.
                for line in analysis["recognitionResults"][0]["lines"]:
                    line_text = " ".join([word['text'] for word in line['words']])
                    text.append(line_text)
                    for word in line['words']:
                        polygons.append( (word["boundingBox"], word["text"]))
                
            angle=analysis["recognitionResults"][0]['clockwiseOrientation']

            lines = analysis["recognitionResults"][0]["lines"]

            # Rotate the bounding boxes, based on the text orientation angle.
            
            origin = (0, 0)
            polygons_rot=[]
            for i in polygons:
                l=i[0]
                it = iter(l)
                nl=[]
                for x in it:
                    point=(x, next(it))
                    c=rotate(origin, point, math.radians(-angle))
                    nl.append(c[0])
                    nl.append(c[1])

                i+=(nl,)
                polygons_rot.append(i)
    
    
    except Exception as e:
        print(e)
        
        text='API response error'
        polygons_rot=None
    
    
    OCR_time=datetime.datetime.now()-start

    return text,OCR_time,polygons_rot

In [528]:
import math

def rotate(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy


In [530]:
filename = r"C:\Users\Mehak.1\Documents\CBSE_digitization\samples\10th_Marksheet_1.jpg"
image_stream=open(filename,"rb").read()

In [531]:
a = OCR_v1(image_stream)[0]

In [532]:
output2=[]
for w in a:
        output2.append(w.upper())
        
print(output2)

['6/17/2020', '10TH_MARKSHEET.JPG', 'BUG: VER Y./2010/', 'S. NO.S.S.E/2010/', 'CENTRAL BOARD OF SECONDARY EDUCATION', '0383213', 'STATEMENT OF SUBJECT WISE PERFORMANCE', 'OLOZ 18IN WEET YOUBATH', 'WITHWWW ALL INDIASECONDARY SCHOOL EXAMINATION, 2010', 'AT4 NAME', 'ASHUTOSH SINGH', '31-195-4145 ROLL NO. 4208593', "HIGH 37 714 MOTHER'S NAME", "FAGT 351 5TH FATHER'S NAME", 'SEEMA SINGH PARIHAR', 'GTA FALE DATE OF BIRTH', 'JITENDRA SINGH PARIHAR', 'PARANICK SCHOOL', '05TH SEPTEMBER NINETEEN HUNDRED NINETY FOUR', '06800 J H AMBANI SCHOOL LODHIVALI CHOWK RAIGAD MR', 'IS FAC ( SIT TT )', 'SUBJECT CODE', 'SUBJECT', 'GRADE', 'GRADE POINT (GP)', '101', 'ENGLISH COMM.', 'AZ', '09', '(NINE)', '085', 'HINDI COURSE-B', 'A1', '10 ( TEN)', '041', 'MATHEMATICS', 'A1', 'O (TEN)', '086', 'SCIENCE', 'A2', '9 (NINE)', '087', 'SOCIAL SCIENCE', 'A1', '10 (TEN)', 'SEND THE FIRE ATREA ( AT SIT TIT U ) / CUMULATIVE GRADE POINT AVERAGE (CGPA) :', '9.6', 'AFTERW/RESULT:', 'ELIGIBLE FOR QUALIFYING CERTIFICATE', 'AB

In [505]:
Examination=''
Result=[]
for x in output2:
    if 'CERTIFICATE EXAMINATION,' in x or 'SECONDARY SCHOOL EXAMINATION' in x:
        Examination+=x

In [506]:
for x in output2:
    for w in x.split(' '):
        if 'PASS' in x:        
            Result.append(x)
            break

In [507]:
Examination

'WITHWWW ALL INDIASECONDARY SCHOOL EXAMINATION, 2010'

In [508]:
Result

[]

In [509]:
add=[i for i, s in enumerate(output2) if 'SCHOOL' in s or 'CERTIFICATE EXAMINATION,' in s]
z1=min(add)
z2=max(add)
fornames=output2[z1+1:z2]

In [511]:
fornames

['AT4 NAME', 'ASHUTOSH SINGH', '31-195-4145 ROLL NO. 4208593', "HIGH 37 714 MOTHER'S NAME", "FAGT 351 5TH FATHER'S NAME", 'SEEMA SINGH PARIHAR', 'GTA FALE DATE OF BIRTH', 'JITENDRA SINGH PARIHAR', 'PARANICK SCHOOL', '05TH SEPTEMBER NINETEEN HUNDRED NINETY FOUR']

In [512]:
Rollno=''
for i in fornames:
    for w in i.split(' '):
        if  re.match('\d{7}',w):
            Rollno=w

In [513]:
fornames

['AT4 NAME', 'ASHUTOSH SINGH', '31-195-4145 ROLL NO. 4208593', "HIGH 37 714 MOTHER'S NAME", "FAGT 351 5TH FATHER'S NAME", 'SEEMA SINGH PARIHAR', 'GTA FALE DATE OF BIRTH', 'JITENDRA SINGH PARIHAR', 'PARANICK SCHOOL', '05TH SEPTEMBER NINETEEN HUNDRED NINETY FOUR']

In [524]:
cutfornames=[]
for i in fornames:
        if 'ROLL' not in i and Rollno not in i and not(re.match('[0-9$]',i)) and 'DATE OF BIRTH' not in i and "MOTHER'S NAME" not in i and "FATHER'S NAME" not in i:
            cutfornames.append(i)            

In [525]:
cutfornames

['AT4 NAME', 'ASHUTOSH SINGH', 'SEEMA SINGH PARIHAR', 'JITENDRA SINGH PARIHAR', 'PARANICK SCHOOL']

In [526]:
Name=cutfornames[0].split()[-2:]
Mother_name=cutfornames[1].split()[-2:]
Father_name=cutfornames[2].split('NAME')[1]
Result=Result[0].split('RESULT')[1]

IndexError: list index out of range

In [518]:
data = {}
data['Examination']=Examination
data['Name'] = Name
data['Roll No'] = Rollno
data["Mother's Name"]=Mother_name
data["Father's Name"] =Father_name
data['Result']=Result

In [519]:
print(data)

{'Examination': 'WITHWWW ALL INDIASECONDARY SCHOOL EXAMINATION, 2010', 'Name': ['AT4', 'NAME'], 'Roll No': '4208593', "Mother's Name": ['ASHUTOSH', 'SINGH'], "Father's Name": '', 'Result': []}
